* ### Họ và tên và MSSV: 
- Trương Chí Toàn - 19127587
- Phan Trung Hiếu - 19127404


---

## Import library

In [1]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns

import warnings
warnings.simplefilter("ignore")

%matplotlib inline
plt.style.use('seaborn-pastel')

## Giới thiệu dữ liệu

Nhóm em xin trình bày về phân tích dữ liệu về elo ranking của vận động viên tennis thuộc giải The Association of Tennis Professionals (ATP)

- Dữ liệu <i>Tennis players ranking based on ELO scores</i> được lấy từ Kaggle tại [đây](https://www.kaggle.com/ramjasmaurya/tennis-players-ranking-based-on-elo-scores?fbclid=IwAR3ReUKj-X3uYBmS2LwP-R1Jc2tLyKSzkar5Y6jcqjXn0bm3XQEzfHQ8Zhg).

- Theo thông tin về license mà tác giả đưa ra trên Kaggle, Ram Jas Maurya cho phép người sử dụng chia sẻ và tùy chỉnh dữ liệu.

In [16]:
# Đọc file csv
tennis_df = pd.read_csv('data.csv', index_col=[0])

## Khám phá dữ liệu

In [17]:
tennis_df.head()

,Age,Elo,HardRaw,ClayRaw,GrassRaw,hard_court_elo_rating,clay_court_elo_rating,grass_court_elo_rating,Peak_Match,Peak_Age,Peak_Elo,Gender,Rank
Player,,,,,,,,,,,,,
Novak�Djokovic,34.5,2185.2,2068.0,2016.3,1942.4,2126.6,2100.8,2063.8,2016 Miami F,28.8,2470.0,Male,1
Daniil�Medvedev,25.9,2166.2,2095.3,1714.1,1723.4,2130.7,1940.1,1944.8,2022 Atp Cup RR,25.9,2175.1,Male,2
Alexander�Zverev,24.7,2141.3,2056.7,1999.1,1671.4,2099.0,2070.2,1906.4,2022 Atp Cup RR,24.7,2157.9,Male,3
Roger�Federer,39.9,2043.0,1936.7,1749.6,1813.4,1989.8,1896.3,1928.2,2007 Dubai F,25.6,2382.0,Male,4
Carlos�Alcaraz,18.5,2029.5,1906.3,1834.4,1441.4,1967.9,1932.0,1735.5,2021 NextGen Finals F,18.5,2029.5,Male,5


# 2. Exploring data (often interleaved with preprocessing)

## How many rows and how many columns?

In [4]:
print("Number of rows: %d\nNumber of columns: %d" % (tennis_df.shape))

Number of rows: 783
Number of columns: 13


## What is the meaning of each row?

Mỗi dòng là thông tin của một vận động viên

## Are there duplicated rows?

In [5]:
have_duplicated_rows = (tennis_df.duplicated().any() == True)
have_duplicated_rows

False

## What is the meaning of each column?

- Player: Tên vận động viên
- Age: Tuổi của họ
- Elo: Điểm elo tổng quát của họ (không tính loại sân)
- HardRaw: Điểm elo khi thi đấu trên sân bê tông
- ClayRaw: Điểm elo khi thi đấu trên sân đất
- GrassRaw: Điểm elo khi thi đấu trên sân cỏ
- Hard_court_elo_rating: Điểm trung bình của Elo tổng và Elo trên sân bê tông
- Clay_court_elo_rating: Điểm trung bình của Elo tổng và Elo trên sân đất
- Grass_court_elo_rating: Điểm trung bình của Elo tổng và Elo trên sân cỏ
- Peak_Match: Giải đấu tham gia khi ở đỉnh cao nhất sự nghiệp
- Peak_Age: Tuổi khi ở đỉnh cao nhất sự nghiệp
- Peak_Elo: Elo khi ở đỉnh cao nhất sự nghiệp
- Gender: Giới tính
- Rank: Thứ hạng

## What is the current data type of each column? Are there columns having inappropriate data types?

In [6]:
col_dtypes = tennis_df.dtypes
col_dtypes

Age                       float64
Elo                       float64
HardRaw                   float64
ClayRaw                   float64
GrassRaw                  float64
hard_court_elo_rating     float64
clay_court_elo_rating     float64
grass_court_elo_rating    float64
Peak_Match                 object
Peak_Age                  float64
Peak_Elo                  float64
Gender                     object
Rank                        int64
dtype: object

Như ta thấy thì cột Peak_Match và Peak_Elo có type là object, nên giờ ta sẽ xem data types của 2 cột này là gì

In [7]:
def open_object_dtype(s):
    dtypes = set()
    dtypes.update(s.apply(lambda x:type(x)).unique())
    return dtypes

open_object_dtype(tennis_df['Peak_Match']),open_object_dtype(tennis_df['Gender'])

({str}, {str})

Vậy cả 2 cột đều có dạng string

## With each numerical column, how are values distributed?

In [8]:
num_col = tennis_df.columns.drop(['Peak_Match','Gender'])
def missing_ratio(col):
    return col.isna().mean() * 100
nume_col_player_df = tennis_df[num_col].agg([missing_ratio,pd.Series.min,pd.Series.max])
nume_col_player_df


,Age,Elo,HardRaw,ClayRaw,GrassRaw,hard_court_elo_rating,clay_court_elo_rating,grass_court_elo_rating,Peak_Age,Peak_Elo,Rank
missing_ratio,0.0,0.0,0.63857,0.383142,20.178799,0.0,0.0,0.0,0.0,0.0,0.0
min,15.5,1032.1,910.20000,908.500000,995.300000,1010.3,970.3,1124.1,15.0,1169.0,1.0
max,42.6,2185.2,2095.30000,2016.300000,1942.400000,2130.7,2100.8,2063.8,35.1,2470.0,396.0


### Are they abnormal?

3 Cột HardRaw, ClayRaw,GrassRaw đều bị thiếu dữ liệu, nhưng mà 3 cột hard_court_elo_rating, clay_court_elo_rating, grass_court_elo_rating tương ứng thì lại không bị thiếu

## With each categorical column, how are values distributed?

In [9]:
cate_col = ['Peak_Match','Gender']
def num_diff_vals(col):
    return len(col.value_counts())
def diff_vals(col):
    return col.value_counts().index.unique().to_numpy()

cate_col_player_df = tennis_df[cate_col].agg([missing_ratio,num_diff_vals,diff_vals])
cate_col_player_df

,Peak_Match,Gender
missing_ratio,0.0,0.0
num_diff_vals,722,2
diff_vals,"[2021 Indian Wells Qual, 2021 Roland Garros Qu...","[Female, Male]"


### Are they abnormal?

Theo em thì hoàn toàn bình thường

# 3. Asking meaningful questions


###### - Giới tính có ảnh hưởng tới tuổi đỉnh cao hay không?
- Giới tính có ảnh hưởng tới trình độ người chơi không?
- Loại sân khác nhau có ảnh hưởng gì tới trình độ người chơi hay không?
- Ở trong khoảng độ tuổi nào thì có nhiều vận động viên đạt đỉnh cao nhất?

# 4. Preprocessing + analyzing data to answer each question

## Does it need to have preprocessing step, and if yes, how does your group preprocess?

Nhóm em sẽ thực hiện tiền xử lý dữ liệu như là:
- Điền lại vào các ô bị trống bằng cách từ công thức Elo = (HardRaw + hard_court_elo_rating)/2 em sẽ truy ngược ra lại giá trị bị thiếu
- Chia dataframe thành 2 dataframe con, chia ra thành nam và nữ

#### Điền giá trị thiếu

In [10]:
def fill_missing_value(col):
    index_of_missing_value = np.argwhere(tennis_df[col].isnull().to_numpy()==True)
    index_of_need_col = int(np.argwhere(tennis_df.columns==col))
    for index in index_of_missing_value:
        tennis_df[col].iloc[index] = tennis_df.iloc[index,index_of_need_col+3] * 2 - tennis_df.Elo[index]
    
fill_missing_value('HardRaw')
fill_missing_value('ClayRaw')
fill_missing_value('GrassRaw')
tennis_df[num_col].agg([missing_ratio,pd.Series.min,pd.Series.max])

,Age,Elo,HardRaw,ClayRaw,GrassRaw,hard_court_elo_rating,clay_court_elo_rating,grass_court_elo_rating,Peak_Age,Peak_Elo,Rank
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,15.5,1032.1,910.2,908.5,995.3,1010.3,970.3,1124.1,15.0,1169.0,1.0
max,42.6,2185.2,2095.3,2016.3,1942.4,2130.7,2100.8,2063.8,35.1,2470.0,396.0


#### Chia dataframe làm 2 phần, nam và nữ

In [34]:
male = tennis_df.Gender.loc[lambda x: x == 'Male'].index[-1]
female = tennis_df.Gender.loc[lambda x: x == 'Female'].index[0]
tennis_male = tennis_df.loc[:male]
tennis_female = tennis_df.loc[female:]
tennis_male

,Age,Elo,HardRaw,ClayRaw,GrassRaw,hard_court_elo_rating,clay_court_elo_rating,grass_court_elo_rating,Peak_Match,Peak_Age,Peak_Elo,Gender,Rank
Player,,,,,,,,,,,,,
Novak�Djokovic,34.5,2185.2,2068.0,2016.3,1942.4,2126.6,2100.8,2063.8,2016 Miami F,28.8,2470.0,Male,1
Daniil�Medvedev,25.9,2166.2,2095.3,1714.1,1723.4,2130.7,1940.1,1944.8,2022 Atp Cup RR,25.9,2175.1,Male,2
Alexander�Zverev,24.7,2141.3,2056.7,1999.1,1671.4,2099.0,2070.2,1906.4,2022 Atp Cup RR,24.7,2157.9,Male,3
Roger�Federer,39.9,2043.0,1936.7,1749.6,1813.4,1989.8,1896.3,1928.2,2007 Dubai F,25.6,2382.0,Male,4
Carlos�Alcaraz,18.5,2029.5,1906.3,1834.4,1441.4,1967.9,1932.0,1735.5,2021 NextGen Finals F,18.5,2029.5,Male,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Daniel�Dutra�Da�Silva,33.5,1233.8,1089.8,1242.9,NaN,1161.8,1238.4,1316.9,2008 Seville CH R32,20.2,1482.5,Male,383
Shintaro�Mochizuki,18.5,1214.1,1163.4,1153.5,1408.0,1188.8,1183.8,1311.1,2021 Wimbledon Qual,18.1,1341.6,Male,384
Nick�Chappell,29.3,1174.0,1212.6,1078.1,1355.2,1193.3,1126.0,1264.6,2021 Potchefstroom 1 CH QF,28.4,1444.1,Male,385


In [ ]:
tennis_temp = tennis_df.copy()
tennis_temp.Age = tennis_temp.Age.apply(lambda x:round(x))
tennis_temp


## Tiền xử lí dữ liệu

#### Loại bỏ các cột không cần thiết

Đầu tiên cần loại bỏ bớt các cột không cần thiết trong bài phân tích này dựa trên những cột được liệt kê phía trên.

Như quan sát, những cột như Joined, Loaned From, Contract Valid Until, Release Clause và cùng với LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB là những cột có giá trị thiếu rất nhiều, và những cột này cũng không cần thiết trong quá trình phân tích cho nên ta sẽ loại bỏ nó và đồng thời cũng loại bỏ một số cột không cần thiết khác.

Dưới đây là mô tả về các cột được sử dụng trong phân tích của dữ liệu FIFA 19 complete player:

+ ID: chỉ số định danh trên FIFA

+ Name: tên cầu thủ.

+ Age: tuổi cầu thủ.

+ Nationality: quốc tịch cầu thủ.

+ Club: câu lạc bộ đang thi đấu.

+ Value: giá trị chuyển nhượng.

+ Wage: lương của cầu thủ.

+ Overall: đánh giá tổng quan.

+ Potential: tiềm năng.

+ Position: vị trí trong sân.

+ Crossing, Finishing, HeadingAccuracy, ShortPassing, Volleys: các chỉ số về tấn công.

+ Dribbling, Curve, FKAccuracy, LongPassing, BallControl: các chỉ số về kĩ năng.

+ Acceleration, SprintSpeed, Agility, Reactions, Balance: các chỉ số về di chuyển.

+ ShotPower, Jumping, Stamina, Strength, LongShots: các chỉ số về sức mạnh.

+ Aggression, Interceptions, Positioning, Vision, Penalties, Composure: các chỉ số về tinh thần.

+ Marking, StandingTackle, SlidingTackle : các chỉ số về phòng thủ.

+ GKKicking, GKPositioning, GKReflexes, GKDiving, GKHandling: các chỉ số về giữ lưới thủ môn.

In [ ]:
football_df = football_df[[
    'ID', 'Name', 'Age', 'Nationality', 'Club', 'Value', 'Wage', 'Overall', 'Potential', 'Position',
    'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys',
    'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl',
    'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 
    'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
    'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
    'Marking', 'StandingTackle', 'SlidingTackle',
    'GKKicking', 'GKPositioning', 'GKReflexes', 'GKDiving', 'GKHandling'
]]

In [ ]:
print("Number of rows: %d\nNumber of columns: %d" % (football_df.shape))

#### Xử lí các giá trị bị thiếu

Đối với những dòng có Club là NaN, có thể là những cầu thủ này chưa có câu lạc bộ cho nên ta sẽ điền 'No Club' vào.

In [ ]:
football_df['Club'].fillna('No Club', inplace=True)

Liệt kê các dòng còn giá trị thiếu để xem xét xử lí.

In [ ]:
football_df[football_df.isnull().any(axis=1)]

Theo quan sát, một số các cầu thủ có nhiều cột chỉ số bị thiếu, cho nên ta sẽ thực hiện loại bỏ các cầu thủ đó và chỉ giữ lại các cầu thủ có ít nhất 11 cột không bị thiếu giá trị, đây là những cột chứa thông tin cá nhân của cầu thủ.

In [ ]:
football_df.dropna(thresh=11, inplace=True)
football_df = football_df.reset_index()

In [ ]:
# Đếm các giá trị null
football_df.isnull().sum()

#### Chuyển đổi Value và Wage từ object về dạng numeric

In [ ]:
football_df.Value.unique(), football_df.Wage.unique()

Các giá trị của Value và Wage là dạng chuỗi với tiền tố là kí hiệu tiền Euro, hậu tố là số lượng chữ số 0 như M = 6 số, K = 3 số

In [ ]:
def convert_value(value):
    """
        Hàm xử lí chuyển chuỗi giá trị sang dạng số.
    """
    value = value[1:]
    if value.endswith('K'):
        return float(value[:-1]) * 1000
    elif value.endswith('M'):
        return float(value[:-1]) * 1000000
    else:
        return float(value)
    
football_df['Value'] = football_df.Value.apply(convert_value)
football_df['Wage'] = football_df.Wage.apply(convert_value)

In [ ]:
main_map_postions = {
    'GK' : ['GK'],
    'FW' : ['RS', 'ST', 'LS', 'CF', 'LF', 'RF', 'LW', 'RW'],
    'MF' : ['LM', 'LCM', 'CM', 'RCM', 'RM', 'LDM', 'CDM', 'RDM', 'LAM', 'CAM', 'RAM'],
    'DF' : ['RB', 'LB', 'LCB', 'LWB', 'RWB', 'CB', 'RCB']
}
def convert_to_main_position(position):
    """
        Hàm gộp các vị trí thi đấu.
    """
    for key, value in main_map_postions.items():
        if position in value:
            return key
    return position

def predict_position(df, model):
    """
        Hàm dự đoán vị trí thi đấu trong sân của cầu thủ.
    """
    if df.Position is np.nan:
        pred_position = model.predict([df.drop(['Position']).values])
        return pred_position[0]
    return df.Position

def train(X_train, y_train):
    """
        Hàm huấn luyện mô hình dự đoán vị trí bằng Logis Regression.
    """
    model = Pipeline([
        ('scale', StandardScaler()),
        ('clf', LogisticRegression(C=100, max_iter=1000, multi_class='multinomial', solver='lbfgs'))
    ])

    # Huấn luyện mô hình
    model.fit(X_train, y_train)
    
    return model

Huấn luyện mô hình dự đoán vị trí thi đấu của cầu thủ, mục tiêu là để lấp vị trí trống của những cầu thủ chưa có vị trí.

In [ ]:
# Tách lọc những đặc trưng cần thiết cho dự đoán
player_skills = football_df[[    'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys',
    'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl',
    'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 
    'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
    'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
    'Marking', 'StandingTackle', 'SlidingTackle',
    'GKKicking', 'GKPositioning', 'GKReflexes', 'GKDiving', 'GKHandling', 'Position']]


player_skills['Position'] = player_skills.Position.apply(convert_to_main_position)
player_skills.sample(10)

In [ ]:
# Chuẩn bị dữ liệu
y_data = player_skills[~player_skills.isnull().any(axis=1)].Position.values
X_data = player_skills[~player_skills.isnull().any(axis=1)].drop(columns=['Position']).values

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2)

In [ ]:
model = train(X_train, y_train)

# Đánh giá mô hình
print("Training accuray:", model.score(X_train, y_train))
print("Validation accuray:", model.score(X_val, y_val))

In [ ]:
# Dự đoán cho cầu thủ đang thiếu vị trí
player_skills['Position'] = player_skills.apply(predict_position, args=(model, ), axis=1)
football_df['Position'] = player_skills['Position']

Hiển thị lại dữ liệu sau khi đã tiền xử lí.

In [ ]:
football_df.sample(5)

In [ ]:
# Mô tả các giữ liệu số
football_df.describe()

## Thống kê dữ liệu

#### 1. Số lượng các cầu thủ thi đấu ở các vị trí trong sân

In [ ]:
print("Tổng số vị trí thi đấu:", football_df.Position.unique().size)
postion_count_df = football_df.Position.value_counts().to_frame().reset_index()
postion_count_df.columns = ['Position', 'Count']
postion_count_df.head(10)

#### 2. Số lượng cầu thủ của các câu lạc bộ

In [ ]:
print("Tổng số câu lạc bộ:", football_df[football_df.Club != 'No Club'].Club.unique().size)
club_count_df = football_df[football_df.Club != 'No Club'].Club.value_counts().to_frame().reset_index()
club_count_df.columns = ['Club', 'Count']
club_count_df.head(10)

#### 3. Số lượng cầu thủ theo từng quốc gia

In [ ]:
print("Tổng số quốc gia:", football_df.Nationality.unique().size)
nationality_count_df = football_df.Nationality.value_counts().to_frame().reset_index()
nationality_count_df.columns = ['Nationality', 'Count']
nationality_count_df.head(10)

#### 4. Thu nhập và giá trị cầu thủ

In [ ]:
valuest_name, value = football_df.iloc[football_df.Value.argmax()][['Name', 'Value']].to_list()
print("Cầu thủ có giá trị cao nhất là {} với {} bảng Anh.".format(valuest_name, int(value)))

wagest_name, wage = football_df.iloc[football_df.Wage.argmax()][['Name', 'Wage']].to_list()
print("Cầu thủ có lương cao nhất là {} với {} bảng Anh.".format(wagest_name, int(wage)))

#### 5. Chỉ số tổng quát và tiềm năng

In [ ]:
print("Chỉ số tổng quát:")
most_overall_name, most_overall = football_df.iloc[football_df.Overall.argmax()][['Name', 'Overall']].to_list()
print("Cầu thủ có chỉ số tổng quát cao nhất là {} với {} điểm.".format(most_overall_name, int(most_overall)))
least_overall_name, least_overall = football_df.iloc[football_df.Overall.argmin()][['Name', 'Overall']].to_list()
print("Cầu thủ có chỉ số tổng quát thấp nhất là {} với {} điểm.".format(least_overall_name, int(least_overall)))

print("Tiềm năng:")
most_potenial_name, most_potential = football_df.iloc[football_df.Potential.argmax()][['Name', 'Potential']].to_list()
print("Cầu thủ có tiềm năng cao nhất là {} với {} điểm.".format(most_potenial_name, int(most_potential)))
least_potential_name, least_potential = football_df.iloc[football_df.Potential.argmin()][['Name', 'Potential']].to_list()
print("Cầu thủ có tiềm năng thấp nhất là {} với {} điểm.".format(least_potential_name, int(least_potential)))

## Phân tích dữ liệu

Một số câu hỏi cần khám phá:

    1. Ai là cầu thủ trẻ có giá trị cao nhất? Và liệu tuổi tác có ảnh hưởng tới lương và giá trị của cầu thủ?
    2. Những câu lạc bộ nào có giá trị cao nhất?
    3. Vị trí thi đấu nào được trả lương cao nhất?
    4. Các vị trí thi đấu trên sân cần có những chỉ số thuộc tính nào?
    5. Quốc gia nào sản sinh ra những cầu thủ tài năng nhất?

#### Câu hỏi 1: Cầu thủ trẻ có giá trị cao nhất?

Đầu tiên ta cần lấy ra Dataframe chỉ gồm các cột Giá trị, lương, chỉ số tổng quát và tiềm năng theo độ tuổi.

In [ ]:
# Lấy ra các cột Value, Wage, Overall, Potential theo Age
age_value_df = football_df.groupby('Age')[['Value', 'Wage', 'Overall', 'Potential']].mean().reset_index()

In [ ]:
# Vẽ 2 biểu đồ đường trong cùng một biểu đồ
fig = plt.figure(figsize=(16, 10))
ax = fig.add_subplot(211)
linechart = ax.plot(age_value_df.Age, age_value_df.Overall, color='b', marker='o', label='Overall')
linechart2 = ax.plot(age_value_df.Age, age_value_df.Potential, color='g', marker='o', label='Potential')

ax.set_xlabel("Age")
ax.set_ylabel("Average of Overall and Potential")
ax.set_title('Chỉ số tổng quát và tiềm năng theo độ tuổi')
fig.legend()
plt.show()

Dựa vào biểu đồ về Chỉ số tổng quát và tiềm năng theo độ tuổi ở phía trên, ta có thể thấy những cầu thủ dưới 25 tuổi là những cầu thủ trẻ có tiềm năng phát triển rất lớn và cao hơn những cầu thủ "lão tướng", còn những cầu thủ lớn hơn 25 tuổi thì chỉ số tổng quát và tiềm năng bằng nhau, tức là các cầu thủ này đã đạt đỉnh phong độ, không thể lên nữa.

Vậy những cầu thủ trẻ liệu có Giá trị và lương cao hơn những cầu thủ được cho là "lão tướng"?

In [ ]:
fig = plt.figure(figsize=(16, 10))

# Vẽ biểu đồ về trung bình giá trị theo tuổi
ax = fig.add_subplot(211)
barchart = ax.bar(age_value_df.Age, age_value_df.Value, color='r', label='Value') # Vẽ bar chart
ax2 = ax.twinx()      # Kết hợp 2 biểu đồ
linechart = ax2.plot(age_value_df.Age, age_value_df.Overall, color='b', marker='o', label='Overall') #Line chart

ax.set_xlabel("Age")
ax.set_ylabel("Average of Value")
ax2.set_ylabel("Average of Overall and Potential")
fig.legend()


# Vẽ biểu đồ về trung bình lương theo tuổi
ax = fig.add_subplot(212)
barchart = ax.bar(age_value_df.Age, age_value_df.Wage, color='r', label='Wage') # Vẽ bar chart
ax2 = ax.twinx()   # Kết hợp 2 biểu đồ
linechart = ax2.plot(age_value_df.Age, age_value_df.Overall, color='b', marker='o', label='Overall') #Vẽ line chart

ax.set_xlabel("Age")
ax.set_ylabel("Average of Wage")
ax2.set_ylabel("Average of Overall and Potential")
plt.show()

Độ tuổi có giá trị và lương cao nhất là khoảng từ 25 cho tới 31, đây là độ tuổi các cầu thủ đều đang đạt đỉnh phong độ của mình.

Càng già tuổi (trên 35 tuổi), cầu thủ nhận lương và có giá trị càng thấp mặc dù chỉ số tổng quát của các cầu thủ này cũng đang rất tốt. Điều này có thể phụ thuộc vào nhiều yếu tố như ít nổi tiếng, sức lực giảm,...

Vậy trong số cầu thủ trẻ đầy tiềm năng, ai là người hiện đang có giá trị cao nhất?

In [ ]:
# Lọc ra danh sách cầu thủ dưới 25 tuổi
valuest_young_player = football_df[football_df.Age <= 25][['Name', 'Age', 'Value']] \
                            .sort_values(['Value', 'Age'], ascending=False)

# Lấy 10 cầu thủ cao nhất
top_valuest_player = valuest_young_player.head(10)
plt.figure(figsize=(16, 5))
sns.barplot(x='Name', y="Value", data=top_valuest_player)
plt.title("Biểu đồ 10 cầu thủ trẻ giá trị cao nhất")
plt.show()

Vậy cầu thủ trẻ dưới 25 tuổi có giá trị cao nhất là P.Dybala, thứ hai là H.Kane và thứ 3 là K.Mbappé

Danh sách những cầu thủ đang đạt phong độ và có giá trị cao nhất

In [ ]:
# Lọc ra danh sách cầu thủ trên 25 tuổi
valuest_well_player = football_df[football_df.Age > 25][['Name', 'Age', 'Value']] \
                            .sort_values(['Value', 'Age'], ascending=False)

# Lấy 10 cầu thủ cao nhất
top_valuest_well_player = valuest_well_player.head(10)
plt.figure(figsize=(16, 5))
sns.barplot(x='Name', y="Value", data=top_valuest_well_player)
plt.title("Biểu đồ 10 cầu thủ có phong độ tốt nhất tại thời điểm hiện tại và có giá trị cao nhất")
plt.show()

Cầu thủ hiện đang thi đấu tốt và có giá trị cao nhất ở thời điểm hiện tại là Neymar JR, L.Messi và K. De Bruyne

#### Câu hỏi 2: Các câu lạc bộ có giá trị cao nhất?

Lấy ra danh sách các câu lạc bộ cùng với tổng Giá trị, lương cùng với trung bình chỉ số tổng quát và tiềm năng của từng câu lạc bộ dựa vào các cầu thủ đang thi đấu cho câu lạc bộ đó.

In [ ]:
attribute_clubs = football_df[['Club', 'Value', 
                               'Wage', 'Overall', 'Potential']].groupby('Club').agg({
                                'Value' : 'sum', 'Wage' : 'sum', 'Overall' : 'mean', 'Potential' : 'mean'
                            }).reset_index()

# Vẽ bar chart 10 câu lạc bộ có Value cao nhất
top_valuest_club = attribute_clubs[['Club', 'Value']].sort_values(['Value'], ascending=False).head(10)
plt.figure(figsize=(16, 15))
plt.subplot(311)
sns.barplot(x='Club', y='Value', data=top_valuest_club)
plt.xticks(rotation=30)
plt.title("Danh sách 10 câu lạc bộ có giá trị cao nhất")

# Vẽ bar chart 10 câu lạc bộ có Overall cao nhất
top_strongest_club = attribute_clubs[['Club', 'Overall']].sort_values(['Overall'], ascending=False).head(10)
plt.figure(figsize=(16, 15))
plt.subplot(312)
sns.barplot(x='Club', y='Overall', data=top_strongest_club)
plt.xticks(rotation=30)
plt.title("Danh sách 10 câu lạc bộ có mạnh nhất")
plt.show()

Ở trên là danh sách 10 câu lạc bộ có giá trị cao nhất và có chỉ số tổng quát cao nhất tính theo trung bình của các cầu thủ đang thi đấu cho câu lạc bộ đó.

Trong đó, Juventus là câu lạc bộ có giá trị và đội hình cao nhất trong số các câu lạc bộ còn lại, chỉ số tổng quát trung bình của đội bóng này là hơn 80 điểm.

Vậy đội hình của câu lạc bộ Juventus gồm những ai?

In [ ]:
juventus_squad = football_df[football_df.Club == 'Juventus'][['Name', 'Age', 'Nationality',
                                                              'Club', 'Value', 'Overall', 'Potential']].reset_index()

display(juventus_squad)

#### Câu hỏi 3: Vị trí thi đấu nào được trả lương cao nhất?

Ta sẽ sử dụng biểu đồ tròn để thể hiện phần trăm lương được trả và số lượng cầu thủ chơi ở vị trí thi đấu đó.

In [ ]:
plt.figure(figsize=(12, 12))

# Lấy ra trung bình của Wage và đếm vị trí theo ID
position_with_value = football_df[['Position', 'Wage', 'ID']].groupby('Position') \
                                                            .agg({'ID':'count', 'Wage':'mean'}) \
                                                            .reset_index()
position_with_value = position_with_value.sort_values(['Wage'], ascending=False)

# Vẽ pie chart của tỉ lệ lương từng vị trí
plt.subplot(121)

plt.pie(position_with_value.Wage, labels=position_with_value.Position, autopct='%1.1f%%')
plt.title('Tỉ lệ lương của từng vị trí')

# Vẽ pie chart của tỉ lệ số lượng cầu thủ theo vị trí
plt.subplot(122)
plt.pie(position_with_value.ID, labels=position_with_value.Position, autopct='%1.1f%%')
plt.title('Tỉ lệ số lượng cầu thủ thi đấu ở vị trí')

plt.show()

Dựa vào biểu đồ ở phía trên, vị trí Tiền đạo (FW) được trả lương cao nhất; Thủ môn là vị trí được trả lương thấp nhất mặc dù thủ môn là vị trí có ít cầu thủ nhất.

Những cầu thủ nào hiện đang chơi những vị trí này tốt nhất?

In [ ]:
best_position_player = football_df[football_df.Overall == 
                                   football_df.groupby('Position')['Overall'].transform(max)]

display(best_position_player[['Name', 'Age', 'Wage', 'Value', 
                              'Position', 'Overall', 'Potential', 'Club', 'Nationality']])

+ Tiền đạo có L.Messi và Cristiano Ronaldo là 2 cầu thủ xuất sắc nhất.

+ Tiền vệ có K. De Bruyne và L. Modric là 2 cầu thủ xuất sắc nhất.

+ Thủ môn De Gea là người giữ lưới xuất sắc nhất.

+ Hậu vệ Sergio Ramos là người xuất sắc nhất.

#### Câu hỏi 4: Các vị trí thi đấu cần có chỉ số nào?

Đầu tiên ta sẽ thực hiện gom nhóm các chỉ số lại, cụ thể như sau:

    - Attacking: Crossing, Finishing, HeadingAccuracy, ShortPassing, Volleys
    - Skill: Dribbling, Curve, FKAccuracy, LongPassing, BallControl
    - Movement: Acceleration, SprintSpeed, Agility, Reactions, Balance
    - Power: ShotPower, Jumping, Stamina, Strength, LongShots
    - Mentality: Aggression, Interceptions, Positioning, Vision, Penalties, Composure
    - Defending: Marking, StandingTackle, SlidingTackle
    - Goalkeeping: GKKicking, GKPositioning, GKReflexes, GKDiving, GKHandling

In [ ]:
# Tạo các cột tương ứng với các nhóm mới
football_df['Attacking'] = football_df[['Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys']].apply(np.mean, axis=1)

football_df['Skill'] = football_df[['Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl']].apply(np.mean, axis=1)

football_df['Movement'] = football_df[['Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance']].apply(np.mean, axis=1)

football_df['Power'] = football_df[['ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots']].apply(np.mean, axis=1)

football_df['Mentality'] = football_df[['Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure']].apply(np.mean, axis=1)

football_df['Defending'] = football_df[['Marking', 'StandingTackle', 'SlidingTackle']].apply(np.mean, axis=1)

football_df['Goalkeeping'] = football_df[[ 'GKKicking',
       'GKPositioning', 'GKReflexes', 'GKDiving', 'GKHandling']].apply(np.mean, axis=1)

Sau khi đã gộp thành công, ta cần tính trung bình giá trị của các cột mới theo Position, đồng thời sắp xếp các giá trị theo thứ tự giảm dần.

In [ ]:
# Gộp lại theo Position và tính trung bình các chỉ số
position_ability = football_df.groupby('Position')[['Attacking', 'Skill', 'Movement',
                                'Power', 'Mentality', 'Defending', 'Goalkeeping']].agg('mean')

# Tách giá trị từng cột ra làm dataframe khác nhau
df_ability = position_ability.transpose().DF.sort_values(ascending=False) \
        .to_frame()
fw_ability = position_ability.transpose().FW.sort_values(ascending=False) \
        .to_frame()
mf_ability = position_ability.transpose().MF.sort_values(ascending=False) \
        .to_frame()
gk_ability = position_ability.transpose().GK.sort_values(ascending=False) \
        .to_frame()

In [ ]:
# Khởi tạo figure
plt.figure(figsize=(16, 10))

# Vẽ barchart hậu vệ (DF)
ax = plt.subplot(221)
sns.barplot(x="DF", y="index", data=df_ability.reset_index(),
            label="Ability", color='r')
ax.set(ylabel="Ability")
ax.set_xlabel('DF', fontsize=18)

# Vẽ barchart tiền đạo (FW)
ax = plt.subplot(222)
sns.barplot(x="FW", y="index", data=fw_ability.reset_index(),
            label="Ability", color='r')
ax.set(ylabel="Ability")
ax.set_xlabel('FW', fontsize=18)

# Vẽ barchart tiền vệ (MF)
ax = plt.subplot(223)
sns.barplot(x="MF", y="index", data=mf_ability.reset_index(),
            label="Ability", color='r')
ax.set(ylabel="Ability")
ax.set_xlabel('MF', fontsize=18)

# Vẽ barchart thủ môn (GK)
ax = plt.subplot(224)
sns.barplot(x="GK", y="index", data=gk_ability.reset_index(),
            label="Ability", color='r')
ax.set(ylabel="Ability")
ax.set_xlabel('GK', fontsize=18)

# Đặt ngang barchart
sns.despine(left=True, bottom=True)

Như 4 biểu đồ phía trên, ta có thể thấy được mỗi vị trí có những chỉ số kĩ năng riêng biệt.

+ Hậu vệ (DF): cần nhất chỉ số phòng thủ (Defending)
+ Tiền đạo (FW): cần nhất chỉ số di chuyển (Movement)
+ Tiền vệ (MF): cần nhất chỉ số di chuyển (Movement)
+ Thủ môn (GK): cần nhất chỉ số giữ lưới (Goalkeeping)

#### Câu hỏi 5: Quốc gia nào có chất lượng cầu thủ tốt nhất?

Trong bộ dữ liệu có sự góp mặt của các cầu thủ tới từ 164 quốc gia, vậy liệu quốc gia nào sẽ có chất lượng cầu thủ tốt nhất, quốc gia nào sẽ có chỉ số tổng quát (Overall) các cầu thủ đạt cao nhất.

Ta sẽ dùng bản đồ địa lí để minh họa theo giá trị trung bình chỉ số tổng quát.

In [ ]:
# Vẽ bản đồ địa lí thể hiện trung bình chỉ số tổng quát Overall của từng quốc gia
nationality_overall_df = football_df.groupby('Nationality')['Overall'].agg('mean').to_frame().reset_index()

fig = px.choropleth(nationality_overall_df, locations='Nationality', 
                    color='Overall', locationmode='country names')

fig.update_layout(title_text='Bản đồ thể hiện trung bình chỉ số tổng quát')
fig.show()

Như ở trên, Châu Âu và khu vực Nam Mỹ có trung bình cầu thủ chất lượng, và Ả Rập Xê-Út là quốc gia có chỉ số cao nhất, lên tới 77 điểm.

## Tổng kết quá trình làm đồ án

### Khó khăn gặp phải

Trong khi thực hiện đồ án, em gặp phải những vấn đề khó khăn trong: 
+ Việc lựa chọn loại biểu đồ phù hợp thể hiện cho các phần trả lời câu hỏi.
+ Xử lí những giá trị bị thiếu một cách hợp lí.
+ Dòng lệnh để truy vấn dữ liệu cho câu trả lời.

### Những điều học được

Nhờ việc thực hiện đồ án này, bàn thân em đã tích lũy được:
+ Quy trình thực hiện phân tích dữ liệu.
+ Hiểu hơn về dữ liệu, cụ thể là lĩnh vực bóng đá.

### Công việc tiếp theo

Sau khi hoàn thành đồ án này, bản thân em muốn mở rộng phần phân tích của mình để phân tích sâu hơn về lĩnh vực này. Cụ thể, phân tích về các lịch thi đấu và xếp hạng, phân tích các giải đấu,...

## Tham khảo

1. [Plotly documentation](https://plotly.com/python/choropleth-maps)
2. [Seaborn documentation](https://seaborn.pydata.org/examples/index.html)
3. [StackOverflow](https://stackoverflow.com/)
4. [Kaggle](https://kaggle.com/)
5. [SoFIFA](https://sofifa.com/)
6. [Matplotlib documentation](https://matplotlib.org/)
7. [Pandas documentation](https://pandas.pydata.org/)